<a href="https://colab.research.google.com/github/benjaminkschmidt/Hate-Speech-Detection-With-BERT/blob/master/BERT_HateSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the necessary packages

In [1]:
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [2]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [3]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-uffbaanh
Created temporary directory: /tmp/pip-req-tracker-jihaxnv2
Created requirements tracker '/tmp/pip-req-tracker-jihaxnv2'
Created temporary directory: /tmp/pip-install-kvk1q7l1
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-707kq0q3
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-jihaxnv2'
    Running setup.py (path:/tmp/pip-req-build-707kq0q3/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.5.1+cu101


    running egg_info
    creating /tmp/pip-req-build-707kq0q3/pip-egg-info/apex.egg-info

# Importing the necessary libraries

In [ ]:
try:
  %tensorflow_version 2.x
  print("Tensorflow Version : " + (tf.__version__))
except Exception:
  print(Exception)
  pass
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from tensorflow.keras import layers
from google.colab import drive
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
# Install a Drive FUSE wrapper.
 # https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth 
auth.authenticate_user()
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
print ('Files in Drive:')
!ls drive/Colab Notebooks
drive.mount('/content/gdrive', force_remount=True)

<class 'Exception'>
E: Package 'python-software-properties' has no installation candidate


Pre-Processing

In [ ]:
import re
import nltk
import torch
#import tensorflow as tf
#import tensorflow_hub as hub
import pandas as pd
#from tensorflow.keras import layers
#from google.colab import drive
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
# Install a Drive FUSE wrapper.
 # https://github.com/astrada/google-drive-ocamlfuse

import re
from sklearn.metrics import cohen_kappa_score
#clean tweets
def preprocess_text(sen):
    sen=str(sen)
    # Removing html tags
    sentence=nltk.sent_tokenize(sen)
    #sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    #Remove unwanted white space
    sentence=re.sub(r'[^\w\s]',' ', sentence )
    #remove punctuation
    sentence= re.sub(r'\s\s+', ' ', sentence)
    #all lowercase 
    sentence=sentence.lower()

    return sentence

#TAG_RE = re.compile(r'<[^>]+>')
#remove html tags
#def remove_tags(text):
    #return TAG_RE.sub('', text)

# Loading the DataFrame

In [ ]:
# -*- coding: utf-8 -*-
"""BERT_Tuned.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1_5tqfnotYZPBcvGbvECyNBtbhRNmZWNJ

# Installating the necessary packages
"""

# Commented out IPython magic to ensure Python compatibility.
# %%writefile setup.sh
#
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex



"""# Importing the necessary libraries"""

# Commented out IPython magic to ensure Python compatibility.

nltk.download('punkt')
"""# Loading the DataFrame"""

# Load the dataset
#from google.colab import drive
#!ls drive/Colab_Notebooks
#df = pd.read_csv("/content/drive/My Drive/Colab_Notebooks/labeled_data.gsheet")
df = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/benjaminkschmidt/Hate-Speech-Detection-With-BERT/master/data/labeled_data.csv", error_bad_lines=False))

print(df.columns)

# Extract the tweet and type column
# df = df[["Tweet", "Type"]]
df = df[["tweet", "class"]]
print(df["class"].value_counts())

# Convert the tweet column to lowercase
print(df["tweet"])
df['tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
#df['tweet'] =  df['tweet'].apply(remove_tags)
print(df['tweet'])




y = df['class']
#print(df.head())
#print(df.shape)
#print((y.value_counts()))

"""# Forming the Training and Testing Sets"""

from sklearn.model_selection import train_test_split
#Wrapper transformer takes a dataframe where the first column is the test data, the second field is the label
X = df["tweet"]
Y = df["class"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

train_df = pd.DataFrame()
test_df = pd.DataFrame()

# Formation of the Training Dataframe
train_df["text"] = X_train
train_df["label"] = y_train
print(train_df.shape)

# Formation of the Testing Dataframe
test_df["text"] = X_test
test_df["label"] = y_test
print(test_df.shape)

#"""# Evaluation on Test Data







EDA

Model Train

In [ ]:
# BERT Model Initialization
!pip install simpletransformers
!pip install tqdm
!pip install tenumerate
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel from simpletransformers (Check what this is)
args = {'reprocess_input_data': True,
      'overwrite_output_dir': True,
      'num_train_epochs': 5}
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, args = args) # 3 output labels
#other options aavilable to tweak, steps specifically
"""# Training the Model"""

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
model.train_model(train_df)


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

def confusion_matrix_cal(labels, pred):
    return confusion_matrix(labels, pred)

result, model_outputs, wrong_predictions, confusion_matrix_kappa = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score, conf_matrix=confusion_matrix_cal)

print("---------- Results ----------")
print(result)
print(model_outputs)
print(wrong_predictions)
print(confusion_matrix_kappa)

# Model Evaluation

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score, conf_matrix=confusion_matrix_cal)
print(result)
print(model_outputs)
print(wrong_predictions)
#print(confusion_matrix_kappa)
cohen_score = cohen_kappa_score(test_df, ,weights='quadratic')
print(cohen_score)
torch.save(model.state_dict(), "model1.pt")

# Training the Model